In [1]:
import datetime
import pandas as pd
import copy
import numpy as np
import sys
import time

def devide_data(user_behavior_data_path, start_day, end_day,validate_or_test_flag=True):
    batch = 0
    second_end_day = (end_day - datetime.timedelta(days=1))    
    start_day_str = str(start_day.date())
    end_day_str = str(end_day.date())
    second_end_day_str = str(second_end_day.date())
    
    name = start_day_str + "to" + second_end_day_str
    #out put file name
    part_data_file_name = "../DataSet/DevideData/"+"part_data"+name+".csv"
    part_data_label_file_name =  "../DataSet/DevideData/"+"part_data"+end_day_str+".csv"   #end day
    part_data_label_filename =  "../DataSet/DevideData/"+"part_data_uicl"+end_day_str+".csv"
    print("output files name,as follows:\n",part_data_file_name,"\n", part_data_label_file_name,"\n", part_data_label_filename)
    #
    start_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print('get uic label data from %s  to %s start time:'%(start_day_str,end_day_str), start_time)
    
    dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d %H')
    
    for df in pd.read_csv(open(user_behavior_data_path, 'r'), 
                      parse_dates=['time'], 
                      index_col = ['time'], 
                      date_parser = dateparse,
                      chunksize = 10000000):  # operation on chunk as the data file is too large
        try:
            part_data = df[start_day_str:second_end_day_str]
            part_data_label = df[end_day_str]
            #print("sangyongjia",df.head(5))
            part_data.to_csv(part_data_file_name,  
                             columns=['user_id','item_id','behavior_type','item_category'],
                             header=False, mode='a')        
            part_data_label.to_csv(part_data_label_file_name,
                             columns=['user_id','item_id','behavior_type','item_category'],
                             header=False, mode='a')

            batch += 1
            print('chunk %d done.' %batch) 

        except StopIteration:
            print("divide the data set finish.")
            break 
    #get uic+label data
    if(validate_or_test_flag):  #train data
        data_file = open(part_data_file_name, 'r')    
        try:
            part_data = pd.read_csv(data_file, header=None, index_col=False)
            #print("read nonbbnbe",part_data)
            part_data.columns = ['time','user_id','item_id','behavior_type','item_category']
            #print("time",part_data['time'])
        finally:
            data_file.close()
        part_data_uic = part_data.drop_duplicates(['user_id', 'item_id', 'item_category'])[['user_id', 'item_id', 'item_category']]
        #print(part_data)
        data_file = open(part_data_label_file_name, 'r')
        try:
            part_data_label = pd.read_csv(data_file,header=None,index_col=False)    
            part_data_label.columns = ['time','user_id','item_id','behavior_type','item_category']
            #print("ddsadadsadadslabel3:",part_data_label,"\n")
        finally:
            data_file.close()

        # uic + label 
        part_data_uic_label_1 = part_data_label[part_data_label['behavior_type'] == 4][['user_id','item_id','item_category']]
        part_data_uic_label_1.drop_duplicates(['user_id','item_id','item_category'], 'last', inplace=True)
        part_data_uic_label_1['label'] = 1
        part_data_uic_label = pd.merge(part_data_uic, 
                                       part_data_uic_label_1,
                                       on=['user_id','item_id','item_category'], 
                                       how='left').fillna(0).astype('int')
        #print(part_data_uic)
        #print(part_data_uic_label_1)
        #print(part_data_uic_label)
        part_data_uic_label.to_csv(part_data_label_filename, index=False)   
        
    else:   #predict data
        data_file = open(part_data_file_name, 'r')    
        try:
            part_data = pd.read_csv(data_file, header=None, index_col=False)
            part_data.columns = ['time','user_id','item_id','behavior_type','item_category']
        finally:
            data_file.close()
        part_data_uic = part_data.drop_duplicates(['user_id', 'item_id', 'item_category'])[['user_id', 'item_id', 'item_category']]
        part_data_uic.to_csv(part_data_label_filename, index=False)   
    end_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print('end time:', end_time)
    """
    start = datetime.datetime(2014,12,15,0,0,0)
    end = datetime.datetime(2014,12,18,0,0,0)
    devide_data("../DataSet/tianchi_fresh_comp_train_user.csv",start,end)
    """